## 模型的保存
tensorflow 使用 [tf.train.Saver](https://www.tensorflow.org/api_docs/python/tf/train/Saver) 接口保存和恢复模型。

最基本的用法如下：
```python
import tensorflow as tf
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my_test_model')
```

保存的结果：
```
my_test_model.data-00000-of-00001
my_test_model.index
my_test_model.meta
checkpoint
```

- `meta` 文件保存了网络的 graph 信息
- `checkpoint` 文件记录了当前最新的是哪一个 checkpoint
- `.data-00000-of-00001` 和 `.index` 文件包含了 weights，biases，gradients 等变量的值

meta 文件可能很大，为了节约磁盘空间，可以在 `saver.save()` 时加上 `write_meta_graph=False` 参数。

构造 saver 时可以向 `tf.train.Saver()` 传递参数指定要保存哪些变量：
```python
import tensorflow as tf
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')
saver = tf.train.Saver([w1]) # 这里只会保存 w1 变量
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.save(sess, 'my_test_model',global_step=1000)
```

## 模型的恢复
模型恢复分为两步
1. 构建网络
2. 恢复变量

构建网络可以有两种方式。

方法一：运行一遍训练时构建网络的代码
```python
import tensorflow as tf

# 构建 graph
w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.Variable(tf.random_normal(shape=[5]), name='w2')

saver = tf.train.Saver()
with tf.Session() as sess:
    latest_ckpt = tf.train.latest_checkpoint('./path/to/ckeckpint_dir')
    saver.restore(sess, latest_ckpt)
```

方法二：直接从 `meta` 中恢复
```python
import tensorflow as tf

saver = tf.train.Saver()
with tf.Session() as sess:
    latest_ckpt = tf.train.latest_checkpoint('./path/to/ckeckpint_dir')
    # 从 meta 文件中恢复网络结构
    saver = tf.train.import_meta_graph('my_test_model-1000.meta')
    saver.restore(sess, latest_ckpt)
```

## 使用恢复好的模型
如果是使用上述的`方法一`恢复的模型，通常可以直接获得 `placeholder` 变量和用来输出的 operation。
如果是按照`方法二`，可以通过 `graph` 来找到网络的输入输出：

```python
import tensorflow as tf

sess=tf.Session()    
# 恢复 graph 和变量
saver = tf.train.import_meta_graph('my_test_model-1000.meta')
saver.restore(sess, tf.train.latest_checkpoint('./path/to/ckeckpint_dir'))

# 获得 graph 对象，通过名字获得 placeholder 对象
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict ={w1:13.0, w2:17.0}

# 获得要进行运算的 operation
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

print sess.run(op_to_restore, feed_dict)
```


参考：
- http://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/
- https://blog.csdn.net/lwplwf/article/details/62419087